In [11]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import time
import pickle
import joblib
from sklearn.metrics import roc_auc_score

from joblib import dump, load
from def_functions import remove_funtion , age_replace , get_d_m_y , \
    separate_genre , song_len, like_language , count_funtion , get_composer_name , \
    get_artist_name , extract_code , get_isrc , calculate_groupby_features 

In [13]:
train = pd.read_csv('data.csv')
train.shape

(7377418, 25)

In [ ]:
q_point = pd.read_csv('q_point_1.csv')
q_point = q_point.drop('id',axis = 1) 


# **Function_1: Prediction Function**

In [8]:
def function_1(q_point):
    start = time.time()
    

    # merge data on test file

    q_point = remove_funtion(q_point)

    ''' fill missing values '''

    q_point['composer'] = q_point['composer'].fillna('no_name')

    q_point['lyricist'] = q_point['lyricist'].fillna('no_name')

    q_point['source_screen_name'] = q_point['source_screen_name'].fillna('not_define')

    q_point['gender'] = q_point['gender'].fillna('not_define')

    # for numeric feature
    q_point['isrc'] = q_point['isrc'].fillna('0')

    # replace missing values with mode 
    q_point['source_system_tab'] = q_point['source_system_tab'].fillna('my library')

    q_point['source_type'] = q_point['source_type'].fillna('local-library')

    q_point['name'] = q_point['name'].fillna('演員')

    q_point['song_length'] = q_point['song_length'].fillna(241812.0)# meadian of song_length 

    q_point['artist_name'] = q_point['artist_name'].fillna('Various Artists')

    q_point['language'] = q_point['language'].fillna('3.0')

    ''' feature extraction '''
    
    # seprate each genre
    q_point = separate_genre(q_point)

    mean = 245120.9693600806
    q_point = song_len(mean,q_point)

    q_point = like_language(q_point)

    # count of composer artist
    composer_count = []
    for i in q_point['composer']:
        s = count_funtion(i,'no_name')
        composer_count.append(s)
    q_point['composer_count'] = composer_count

    # count of lyricist artist
    lyricist_count = []
    for i in q_point['lyricist']:
        s = count_funtion(i,'no_name')
        lyricist_count.append(s)
    q_point['lyricist_count'] = lyricist_count

    # get first composer name
    q_point = get_composer_name(q_point)

    # get first artist name
    q_point = get_artist_name(q_point)

    # extract features from isrc: country_code , regi_code , year , designation_code
    q_point = get_isrc(q_point)

    # group by features
    q_point = calculate_groupby_features(q_point)

    # drop feature which have decrease performance
    drop_list  = ['city','composer_count','country_code ', 'eight_genre',
                  'expire_month', 'five_genre','four_genre', 'gender',
                  'isrc', 'one_genre','regi_day', 'seven_genre', 'six_genre', 'three_genre']
    q_point = q_point.drop(drop_list , axis = 1)

    cat_features = ['msno','song_id', 'source_system_tab', 'source_screen_name',
                    'source_type','registered_via', 'regi_month','regi_year', 
                    'expire_day', 'expire_year',
                    'language', 'two_genre','composer_first_name',
                    'first_artist_name','regi_code', 'year','designation_code',
                    'artist_name','lyricist','composer','name']

    q_point[cat_features] = q_point[cat_features].astype(str) 
    
    # label encoding
    label_dic = ['song_id.pkl','source_system_tab.pkl','source_screen_name.pkl',
                'source_type.pkl','registered_via.pkl','regi_month.pkl','regi_year.pkl',
                'expire_day.pkl','expire_year.pkl','language.pkl','two_genre.pkl',
                'composer_first_name.pkl','first_artist_name.pkl','regi_code.pkl','year.pkl',
                'designation_code.pkl','msno.pkl','artist_name.pkl','composer.pkl',
                'lyricist.pkl']
    
    label_dic = os.listdir(r"G:\Project 1\Music Recommandation Systam\Deployment\label_dicts\old")

    for name in label_dic:
        # load pickle file of dict for labels
        #print(name)
        pkl_file = open(r'G:\Project 1\Music Recommandation Systam\Deployment\label_dicts\old\{}'.format(name), 'rb')
        le = pickle.load(pkl_file) 
        
        col_name = name[:-7]
        # replace key(categories) with there values(labels)
        q_point[col_name] = q_point[col_name].astype(str)
        q_point[col_name] = q_point[col_name].map(le)
        #b.close()
        #pkl_file.close()
        #print(q_point)

    name = 'name.pickle'
    # label encoding
    with open(r"G:\Project 1\Music Recommandation Systam\Deployment\label_dicts\new\name.pickle", 'rb') as handle:
        b = pickle.load(handle)
    col_name = name[:-7]
    # replace key(categories) with there values(labels)
    q_point[col_name] = q_point[col_name].astype(str)
    q_point[col_name] = q_point[col_name].map(b)
    handle.close()
    
    # standar scaler
    std = os.listdir(r"G:\Project 1\Music Recommandation Systam\Deployment\std_scaler")
    for name in std:
        # load standard scaler
        scaler = load(r"G:\Project 1\Music Recommandation Systam\Deployment\std_scaler\{}".format(name))
        col_name = name[:-15]
        q_point[col_name] = q_point[col_name].astype(float)
        q_point[col_name] = scaler.transform(q_point[col_name].values.reshape(-1,1))
    
    prob = ['source_screen_name.pickle', 'source_type.pickle']
    #print('std Done.')
    for name in prob:
        with open(r"G:\Project 1\Music Recommandation Systam\Deployment\prob\{}".format(name), 'rb') as handle:
            b = pickle.load(handle)
        col_name = name[:-7]
        q_point['prob_'+col_name] = q_point[col_name].map(b)
    #print('prob Done')
    lgbm = joblib.load(r"G:\Project 1\Music Recommandation Systam\Deployment\best_model\lgbm.pkl")
    #print('start prediction.')
    pred = lgbm.predict(q_point)

    #print('Run Time :',(time.time() - start) , 'sec' )

    return pred

# **Prediction**

In [14]:
start = time.time()

prediction = function_1(q_point)
print()
print("Predicted Class is {}.".format(prediction[0]))
print('Run Time :',time.time() - start)


Predicted Class is 1.
Run Time : 4.499501705169678


# **Function_2 : Compute Performance Matric**

In [89]:
def function_2(q_point,y_label):
    start = time.time()
    # merge data on test file
    member = pd.read_csv('/content/drive/MyDrive/Case_study/new_member.csv')
    song_info = pd.read_csv('/content/drive/MyDrive/Case_study/csv_files/songs_info.csv')

    test_member = q_point.merge(member , how = 'left' , on = 'msno')
    q_point = test_member.merge(songs_info, how='left', on='song_id')
    q_point = remove_funtion(q_point)

    ''' fill missing values '''

    q_point['composer'] = q_point['composer'].fillna('no_name')

    q_point['lyricist'] = q_point['lyricist'].fillna('no_name')

    q_point['source_screen_name'] = q_point['source_screen_name'].fillna('not_define')

    q_point['gender'] = q_point['gender'].fillna('not_define')

    # for numeric feature
    q_point['isrc'] = q_point['isrc'].fillna('0')

    # replace missing values with mode 
    q_point['source_system_tab'] = q_point['source_system_tab'].fillna('my library')

    q_point['source_type'] = q_point['source_type'].fillna('local-library')

    q_point['name'] = q_point['name'].fillna('演員')

    q_point['song_length'] = q_point['song_length'].fillna(241812.0)# meadian of song_length 

    q_point['artist_name'] = q_point['artist_name'].fillna('Various Artists')

    q_point['language'] = q_point['language'].fillna('3.0')

    ''' feature extraction '''
    
    # seprate each genre
    q_point = separate_genre(q_point)

    mean = 245120.9693600806
    q_point = song_len(mean,q_point)

    q_point = like_language(q_point)

    # count of composer artist
    composer_count = []
    for i in q_point['composer']:
        s = count_funtion(i,'no_name')
        composer_count.append(s)
    q_point['composer_count'] = composer_count

    # count of lyricist artist
    lyricist_count = []
    for i in q_point['lyricist']:
        s = count_funtion(i,'no_name')
        lyricist_count.append(s)
    q_point['lyricist_count'] = lyricist_count

    # get first composer name
    q_point = get_composer_name(q_point)

    # get first artist name
    q_point = get_artist_name(q_point)

    # extract features from isrc: country_code , regi_code , year , designation_code
    q_point = get_isrc(q_point)

    # group by features
    q_point = calculate_groupby_features(q_point)

    # drop feature which have decrease performance
    drop_list  = ['city','composer_count','country_code ', 'eight_genre',
                  'expire_month', 'five_genre','four_genre', 'gender',
                  'isrc', 'one_genre','regi_day', 'seven_genre', 'six_genre', 'three_genre']
    q_point = q_point.drop(drop_list , axis = 1)

    cat_features = ['msno','song_id', 'source_system_tab', 'source_screen_name',
                    'source_type','registered_via', 'regi_month','regi_year', 
                    'expire_day', 'expire_year',
                    'language', 'two_genre','composer_first_name',
                    'first_artist_name','regi_code', 'year','designation_code',
                    'artist_name','lyricist','composer','name']

    q_point[cat_features] = q_point[cat_features].astype(str) 

    # label encoding
    label_dic = ['song_id.pkl','source_system_tab.pkl','source_screen_name.pkl',
                'source_type.pkl','registered_via.pkl','regi_month.pkl','regi_year.pkl',
                'expire_day.pkl','expire_year.pkl','language.pkl','two_genre.pkl',
                'composer_first_name.pkl','first_artist_name.pkl','regi_code.pkl','year.pkl',
                'designation_code.pkl','msno.pkl','artist_name.pkl','composer.pkl',
                'lyricist.pkl']
    for name in label_dic:
        # load pickle file of dict for labels
        pkl_file = open('/content/drive/MyDrive/Case_study/label_dicts/new/{}'.format(name), 'rb')
        le = pickle.load(pkl_file) 
        
        col_name = name[:-4]
        #print(col_name)
        # replace key(categories) with there values(labels)
        q_point[col_name] = le.transform(q_point[col_name])
        #b.close()
        pkl_file.close()

    name = 'name.pickle'
    # label encoding
    with open('/content/drive/MyDrive/Case_study/label_dicts/new/name.pickle', 'rb') as handle:
        b = pickle.load(handle)
    col_name = name[:-7]
    # replace key(categories) with there values(labels)
    q_point[col_name] = q_point[col_name].astype(str)
    q_point[col_name] = q_point[col_name].map(b)
    handle.close()

    # standar scaler
    std = os.listdir('/content/drive/MyDrive/Case_study/std_scaler')
    for name in std:
        # load standard scaler
        scaler = load('/content/drive/MyDrive/Case_study/std_scaler/{}'.format(name))
        col_name = name[:-15]
        q_point[col_name] = q_point[col_name].astype(float)
        q_point[col_name] = scaler.transform(q_point[col_name].values.reshape(-1,1))
    
    #find conditional probability of both features
    prob = ['source_screen_name.pickle', 'source_type.pickle']
    for name in prob:
        with open('/content/drive/MyDrive/Case_study/prob/{}'.format(name), 'rb') as handle:
            b = pickle.load(handle)
        col_name = name[:-7]
        q_point['prob_'+col_name] = q_point[col_name].map(b)

    # for auc score we need probability of class
    # for improve performance we take average of both model
    stacking = joblib.load('/content/drive/MyDrive/Case_study/best_model/stacking.pkl')
    lgbm = joblib.load('/content/drive/MyDrive/Case_study/best_model/lgbm.pkl')

    pred1 = stacking.predict_proba(q_point)
    pred2 = lgbm.predict_proba(q_point)

    pred1 = pred1[:,1]
    pred2 = pred2[:,1]

    pred = (pred1 + pred2) / 2

    print('AUC Score  : ',roc_auc_score(y_label , pred))
    print('Run Time   :',(time.time() - start) , 'sec' )

    return pred

In [70]:
# take sample from train data 
test_data = train.sample(100)
y_label   = test_data['target']
test_data = test_data.drop('target',axis = 1)

In [71]:
pred = function_2(test_data , y_label)

AUC Score  :  0.6398559423769508
Run Time   : 99.38778257369995 sec
